### 2. 练习
#### 【练习一】 继续使用上一章的药物数据集：

In [4]:
import pandas as pd

In [5]:
pd.read_csv('data/Drugs.csv').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### (a) 现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：
![avatar](picture/drug_pic.png)
#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [6]:
data = pd.read_csv('data/Drugs.csv',index_col=['State','COUNTY']).sort_index()
data

YYYY  SubstanceName  DrugReports
State COUNTY                                   
KY    ADAIR    2010      Methadone            1
      ADAIR    2010    Hydrocodone            6
      ADAIR    2011      Oxycodone            4
      ADAIR    2011  Buprenorphine            3
      ADAIR    2011       Morphine            2
...             ...            ...          ...
WV    WYOMING  2013  Buprenorphine            1
      WYOMING  2014    Hydrocodone            1
      WYOMING  2014      Oxycodone            5
      WYOMING  2016    Hydrocodone            1
      WYOMING  2017  Buprenorphine            1

[24062 rows x 3 columns]

In [7]:
result = pd.pivot_table(data,index=['State','COUNTY','SubstanceName']
                 ,columns='YYYY'
                 ,values='DrugReports',fill_value='-').reset_index().rename_axis(columns={'YYYY':''})
result

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6,4,24,7,7,11,7,1
6210,WV,WOOD,Tramadol,-,-,-,-,1,-,4,3
6211,WV,WYOMING,Buprenorphine,-,1,1,1,-,-,-,1
6212,WV,WYOMING,Hydrocodone,1,5,-,-,1,-,1,-


In [14]:
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY'
                                    ,'SubstanceName']).reset_index().drop(columns='index')
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols].astype({'DrugReports':'int','YYYY':'int'})
result2

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2
...,...,...,...,...,...
24057,WV,WYOMING,2013,Oxycodone,12
24058,WV,WYOMING,2014,Hydrocodone,1
24059,WV,WYOMING,2014,Oxycodone,5
24060,WV,WYOMING,2016,Hydrocodone,1


In [15]:
data_tidy = data.reset_index().sort_values(by=result2.columns[:4].tolist()).reset_index().drop(columns='index')
data_tidy.equals(result2)

False

#### 【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：

In [10]:
pd.read_csv('data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


#### (a) 现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：
![avatar](picture/earthquake_pic.png)
#### (b) 现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [16]:
df = pd.read_csv('data/Earthquake.csv')
df = df.sort_values(by=df.columns.tolist()[:3]).sort_index(axis=1).reset_index().drop(columns='index')
df.head()

,方向,日期,时间,深度,烈度,经度,维度,距离
0,south_east,1912.08.09,12:29:00 AM,16.0,6.7,27.2,40.6,4.3
1,south_west,1912.08.10,12:23:00 AM,15.0,6.0,27.1,40.6,2.0
2,south_west,1912.08.10,12:30:00 AM,15.0,5.2,27.1,40.6,2.0
3,south_east,1912.08.11,12:19:04 AM,30.0,4.9,27.2,40.6,4.3
4,south_west,1912.08.11,12:20:00 AM,15.0,4.5,27.1,40.6,2.0


In [18]:
result = pd.pivot_table(df,index=['日期','时间','维度','经度']
            ,columns='方向'
            ,values=['烈度','深度','距离'],fill_value='-').stack(level=0).rename_axis(index={None:'地震参数'})
result.head()

方向                                    east north north_east north_west south  \
日期         时间          维度   经度   地震参数                                          
1912.08.09 12:29:00 AM 40.6 27.2 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   
                                 距离      -     -          -          -     -   
1912.08.10 12:23:00 AM 40.6 27.1 深度      -     -          -          -     -   
                                 烈度      -     -          -          -     -   

方向                                    south_east south_west west  
日期         时间          维度   经度   地震参数                             
1912.08.09 12:29:00 AM 40.6 27.2 深度           16          -    -  
                                 烈度          6.7          -    -  
                                 距离          4.3          -    -  
1912.08.10 12:23:00 AM 40.6 27.1 深度            -         15    -  
                                 烈度            -          6    -

In [19]:
df_result = result.unstack().stack(0)[(~(result.unstack().stack(0)=='-')).any(1)].reset_index()
df_result.columns.name=None
df_result = df_result.sort_index(axis=1).astype({'深度':'float64','烈度':'float64','距离':'float64'})
df_result.head()

,方向,日期,时间,深度,烈度,经度,维度,距离
0,south_east,1912.08.09,12:29:00 AM,16.0,6.7,27.2,40.6,4.3
1,south_west,1912.08.10,12:23:00 AM,15.0,6.0,27.1,40.6,2.0
2,south_west,1912.08.10,12:30:00 AM,15.0,5.2,27.1,40.6,2.0
3,south_east,1912.08.11,12:19:04 AM,30.0,4.9,27.2,40.6,4.3
4,south_west,1912.08.11,12:20:00 AM,15.0,4.5,27.1,40.6,2.0


In [20]:
df_result.astype({'深度':'float64','烈度':'float64','距离':'float64'},copy=False).dtypes

方向     object
日期     object
时间     object
深度    float64
烈度    float64
经度    float64
维度    float64
距离    float64
dtype: object

In [21]:
df.equals(df_result)

True